<a href="https://colab.research.google.com/github/jmcaussade/Analitica_textual_con_ML_PF/blob/main/Proyecto_Final_Dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
!pip install tqdm
!pip install datasets
!pip install langdetect
!pip install num2words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filen

In [2]:
import pandas as pd
import os
import re
import re
from num2words import num2words
from tqdm import tqdm
from datasets import load_dataset
from langdetect import detect
from transformers import GPT2Tokenizer

## Para trabajar con un pedazo del dataset orginal

In [3]:
## SAMPLE DEL DATASET
dataset = load_dataset("somosnlp/recetas-cocina")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/158 [00:00<?, ?B/s]

dataset.csv:   0%|          | 0.00/39.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28238 [00:00<?, ? examples/s]

In [4]:
df = dataset['train'].to_pandas()

# Sample the first 5000 rows from the DataFrame
#df = df.head(5000)

# Show the first few rows of the sampled data
df.head()

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25


## DATASET COMLETO

In [ ]:
## DATASET COMPLETO
#df = pd.read_csv("hf://datasets/CodeKapital/CookingRecipes/Data.csv")


In [5]:
df

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25
...,...,...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,https://www.recetasgratis.net/receta-de-berenj...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...,b0a6454b-a023-44af-a189-92dee8948490
28234,Puré de patatas con queso y huevo,https://www.recetasgratis.net/receta-de-pure-d...,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p...",6e52da68-9496-47de-b85c-7be7fd0b6ac2
28235,Rollitos de calabacín rellenos de jamón y queso,https://www.recetasgratis.net/receta-de-rollit...,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...,22412360-37fe-47de-9401-d3f1e95fe4e7
28236,Calabacín relleno de quinoa,https://www.recetasgratis.net/receta-de-calaba...,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...,eedfabb6-d14f-46e4-a34f-dc07b9015636


In [6]:
df = df.drop(columns=["url", "uuid"])
df

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...
28234,Puré de patatas con queso y huevo,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p..."
28235,Rollitos de calabacín rellenos de jamón y queso,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...
28236,Calabacín relleno de quinoa,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...


In [7]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
title          5597
ingredients       1
steps             0
dtype: int64


In [8]:
# Drop rows where 'title'  y "ingredients" is null
df = df.dropna(subset=['title'])
df = df.dropna(subset=['ingredients'])


In [9]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 3223


In [10]:
# Drop duplicate rows
df = df.drop_duplicates()

In [11]:
print(df.dtypes)

title          object
ingredients    object
steps          object
dtype: object


In [12]:
df

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...
28234,Puré de patatas con queso y huevo,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p..."
28235,Rollitos de calabacín rellenos de jamón y queso,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...
28236,Calabacín relleno de quinoa,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...


In [13]:
df.shape[0]

19417

#Cambiar números a texto

In [ ]:
import inflect

In [47]:
from num2words import num2words  # Ensure this is installed: pip install num2words

# Function to convert numbers to words in Spanish
def convert_number_to_spanish(text):
    if not isinstance(text, str):
        return text  # Return original value if not a string

    # Split string into list of words
    temp_str = text.split()
    # Initialize an empty list for processed words
    new_string = []

    for word in temp_str:
        # If word is a digit, convert the digit to words in Spanish
        if word.isdigit():
            temp = num2words(int(word), lang='es')
            new_string.append(temp)
        else:
            # Append the word as it is
            new_string.append(word)

    # Join the words in new_string to form a single string
    return ' '.join(new_string)

# Apply the function to the 'steps' column
df['converted_steps'] = df['steps'].apply(convert_number_to_spanish)


In [49]:
df.head(1000)

,title,ingredients,steps,converted_steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...","En una olla grande ponga la carne, el caldo, e..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...
7490,Receta FÁCIL y DELICIOSA,12 discos de empanada 500 gramos de osobuco 1 ...,1 Para empezar con la receta de las empanadas ...,uno Para empezar con la receta de las empanada...
7491,Receta FÁCIL y SALUDABLE,6 tapas de empanadas vegetarianas o veganas 1 ...,1 Para empezar con la receta de las empanadas ...,uno Para empezar con la receta de las empanada...
7492,Canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 Para empezar con la receta de las canastitas...,uno Para empezar con la receta de las canastit...
7493,Tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 Para empezar con la receta de tacos de canas...,uno Para empezar con la receta de tacos de can...


#Cambiar fracciones a texto

In [50]:
# Mapping of Unicode fractions to words
unicode_fraction_map = {
    '¼': 'un cuarto',
    '½': 'un medio',
    '¾': 'tres cuartos',
    '⅓': 'un tercio',
    '⅔': 'dos tercios',
    '⅕': 'un quinto',
    '⅖': 'dos quintos',
    '⅗': 'tres quintos',
    '⅘': 'cuatro quintos',
    '⅙': 'un sexto',
    '⅚': 'cinco sextos',
    '⅛': 'un octavo',
    '⅜': 'tres octavos',
    '⅝': 'cinco octavos',
    '⅞': 'siete octavos',
}

# Function to replace fractions in text
def convert_unicode_fractions(text):
    if not isinstance(text, str):
        return text  # Skip non-string inputs

    # Replace each Unicode fraction with its text equivalent
    for unicode_fraction, word in unicode_fraction_map.items():
        text = text.replace(unicode_fraction, word)

    return text


# Apply the function to the 'steps' column
df['converted_steps_2'] = df['converted_steps'].apply(convert_unicode_fractions)



In [51]:
df.head(1000)


,title,ingredients,steps,converted_steps,converted_steps_2
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...","En una olla grande ponga la carne, el caldo, e...","En una olla grande ponga la carne, el caldo, e..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...,...
7490,Receta FÁCIL y DELICIOSA,12 discos de empanada 500 gramos de osobuco 1 ...,1 Para empezar con la receta de las empanadas ...,uno Para empezar con la receta de las empanada...,uno Para empezar con la receta de las empanada...
7491,Receta FÁCIL y SALUDABLE,6 tapas de empanadas vegetarianas o veganas 1 ...,1 Para empezar con la receta de las empanadas ...,uno Para empezar con la receta de las empanada...,uno Para empezar con la receta de las empanada...
7492,Canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 Para empezar con la receta de las canastitas...,uno Para empezar con la receta de las canastit...,uno Para empezar con la receta de las canastit...
7493,Tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 Para empezar con la receta de tacos de canas...,uno Para empezar con la receta de tacos de can...,uno Para empezar con la receta de tacos de can...


In [45]:
df = df.drop(columns=['converted_steps'])


#Cambiar todo a español

In [ ]:
# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"  # Handle cases where detection might fail

# Apply language detection to the column
df['language'] = df['text'].apply(detect_language)

# Filter rows based on language
spanish_rows = df[df['language'] == 'es']
english_rows = df[df['language'] == 'en']

print(df)
print("\nRows in Spanish:")
print(spanish_rows)
print("\nRows in English:")
print(english_rows)


In [ ]:
## Average number of words per recipie
sum = 0
for index, row in df.iterrows():  # Iterate over rows using iterrows()
    word_count = 0
    for sentence in row['directions']:  # Access the 'directions' column
        word_count += len(sentence.split())  # Count words in each sentence
    sum += word_count

average_word_count = sum / df.shape[0]  # Calculate the average
print(average_word_count)


216.9418


In [ ]:
r = df.iloc[0]

In [ ]:
print(r)

Unnamed: 0                                                     0
title                                        No-Bake Nut Cookies
ingredients    ["1 c. firmly packed brown sugar", "1/2 c. eva...
directions     ["In a heavy 2-quart saucepan, mix brown sugar...
Name: 0, dtype: object


In [ ]:
## Clean recipies

In [ ]:
for index, row in df.iterrows():
    # Update 'directions' with all replacements
    updated_directions = row["directions"].replace('"', '').replace('[', '').replace(']', '')
    df.at[index, "directions"] = updated_directions

    # Update 'ingredients' with all replacements
    updated_ingredients = row["ingredients"].replace('"', '').replace('[', '').replace(']', '')
    df.at[index, "ingredients"] = updated_ingredients

# Now the DataFrame should have updated values for both columns
print(df[["directions", "ingredients"]])

                                             directions  \
0     In a heavy 2-quart saucepan, mix brown sugar, ...   
1     Place chipped beef on bottom of baking dish., ...   
2     In a slow cooker, combine all ingredients. Cov...   
3     Boil and debone chicken., Put bite size pieces...   
4     Combine first four ingredients and press in 13...   
...                                                 ...   
4995  Dissolve jello in boiling water., Stir in froz...   
4996  Mix together the flour, nuts, sugar, salt, sod...   
4997  Place all ingredients in a Dutch oven and cove...   
4998  Blend all ingredients by hand., Chill and serv...   
4999  Cream shortening and sugar until light and flu...   

                                            ingredients  
0     1 c. firmly packed brown sugar, 1/2 c. evapora...  
1     1 small jar chipped beef, cut up, 4 boned chic...  
2     2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. cr...  
3     1 large whole chicken, 2 (10 1/2 oz.) cans chi...  
4

In [ ]:
x = df.at[0, "ingredients"]
print(x[0])

1


In [ ]:
df

In [ ]:
#To lower case
df["directions"] = df["directions"].str.lower()
df["ingredients"] = df["ingredients"].str.lower()


In [ ]:
import re

#regular expression to keep slashes (/) and parentheses () characters
df["directions"] = df["directions"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s/()]', '', x))
df["ingredients"] = df["ingredients"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s/()]', '', x))



In [ ]:
#Remove extra white spaces
df["directions"] = df["directions"].str.strip().replace(r'\s+', ' ', regex=True)
df["ingredients"] = df["ingredients"].str.strip().replace(r'\s+', ' ', regex=True)


In [ ]:
df

#Tokenization

In [ ]:
from transformers import GPT2Tokenizer

In [ ]:
df

In [ ]:
# Drop the directions_tokens and ingredients_tokens columns
#df.drop(columns=['directions_tokens', 'ingredients_tokens', 'directions_lenght','ingredients_lenght'], inplace=True)
df.drop(columns=['directions_length','ingredients_length'], inplace=True)



In [ ]:
# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Assuming df is your DataFrame with 'directions' and 'ingredients' columns
# Tokenize the 'directions' and 'ingredients' columns

df['directions_tokens'] = df['directions'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df['ingredients_tokens'] = df['ingredients'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# Check the result
print(df[['directions_tokens', 'ingredients_tokens']].head())


                                   directions_tokens  \
0  [259, 257, 4334, 362, 36008, 10746, 6839, 5022...   
1  [5372, 442, 3949, 12023, 319, 4220, 286, 16871...   
2  [259, 257, 3105, 37171, 12082, 477, 9391, 3002...   
3  [2127, 346, 290, 1915, 505, 9015, 1234, 13197,...   
4  [24011, 500, 717, 1440, 9391, 290, 1803, 287, ...   

                                  ingredients_tokens  
0  [16, 269, 14245, 11856, 7586, 7543, 352, 14, 1...  
1  [16, 1402, 17379, 442, 3949, 12023, 2005, 510,...  
2  [17, 1467, 15649, 279, 10025, 12912, 11676, 35...  
3  [16, 1588, 2187, 9015, 362, 838, 352, 14, 17, ...  
4  [16, 269, 26636, 9215, 513, 14, 19, 269, 7933,...  


In [ ]:
df

In [ ]:
# Check if there are null values in 'directions_tokens' and 'ingredients_tokens'
null_values = df[['directions_tokens', 'ingredients_tokens']].isnull().sum()

# Print the result
print(null_values)

directions_tokens     0
ingredients_tokens    0
dtype: int64


In [ ]:
# Check if there are any NaN values in the directions_tokens and ingredients_tokens columns
nan_check = df[['directions_tokens', 'ingredients_tokens']].isna().sum()

print(nan_check)


directions_tokens     0
ingredients_tokens    0
dtype: int64


In [ ]:
# Check the first few tokenized samples
print(df['directions_tokens'].head())
print(df['ingredients_tokens'].head())

# Decode a sample tokenized row to see the tokens converted back to text
sample_row = df['directions_tokens'].iloc[0]
decoded_sample = tokenizer.decode(sample_row)
print(decoded_sample)


In [ ]:
# Check for empty token lists
empty_directions = df[df['directions_tokens'].apply(lambda x: len(x) == 0)]
empty_ingredients = df[df['ingredients_tokens'].apply(lambda x: len(x) == 0)]

print(f"Empty directions tokens: {len(empty_directions)}")
print(f"Empty ingredients tokens: {len(empty_ingredients)}")


In [ ]:
# Check the lengths of the tokenized sequences
df['directions_length'] = df['directions_tokens'].apply(len)
df['ingredients_length'] = df['ingredients_tokens'].apply(len)

# View summary statistics of token lengths
print(df[['directions_length', 'ingredients_length']].describe())


In [ ]:
# Manually set the padding token ID (typically, you can use an unused token ID)
tokenizer.pad_token = tokenizer.eos_token  # Use end-of-sequence token as pad token
tokenizer.pad_token_id = tokenizer.encode(tokenizer.pad_token)[0]  # Get token ID for pad_token

# Check if the pad_token_id is set correctly
print(f"Pad Token ID: {tokenizer.pad_token_id}")


Pad Token ID: 50256


In [ ]:
# Padding to max length
max_length = 512  # Define your desired maximum length

# Apply padding to both directions and ingredients tokens
df['directions_tokens'] = df['directions_tokens'].apply(lambda x: x[:max_length] + [tokenizer.pad_token_id] * (max_length - len(x)) if len(x) < max_length else x[:max_length])
df['ingredients_tokens'] = df['ingredients_tokens'].apply(lambda x: x[:max_length] + [tokenizer.pad_token_id] * (max_length - len(x)) if len(x) < max_length else x[:max_length])

# Check if padding is working as expected
print(df[['directions_tokens', 'ingredients_tokens']].head())



                                   directions_tokens  \
0  [259, 257, 4334, 362, 36008, 10746, 6839, 5022...   
1  [5372, 442, 3949, 12023, 319, 4220, 286, 16871...   
2  [259, 257, 3105, 37171, 12082, 477, 9391, 3002...   
3  [2127, 346, 290, 1915, 505, 9015, 1234, 13197,...   
4  [24011, 500, 717, 1440, 9391, 290, 1803, 287, ...   

                                  ingredients_tokens  
0  [16, 269, 14245, 11856, 7586, 7543, 352, 14, 1...  
1  [16, 1402, 17379, 442, 3949, 12023, 2005, 510,...  
2  [17, 1467, 15649, 279, 10025, 12912, 11676, 35...  
3  [16, 1588, 2187, 9015, 362, 838, 352, 14, 17, ...  
4  [16, 269, 26636, 9215, 513, 14, 19, 269, 7933,...  


In [ ]:
# Check the pad_token_id to ensure it's valid
print(f"Pad Token ID: {tokenizer.pad_token_id}")


Pad Token ID: 50256


In [ ]:
# Check for invalid token IDs (e.g., negative or zero values)
invalid_directions = df[df['directions_tokens'].apply(lambda x: any(i < 0 for i in x))]
invalid_ingredients = df[df['ingredients_tokens'].apply(lambda x: any(i < 0 for i in x))]

print(f"Invalid directions tokens: {len(invalid_directions)}")
print(f"Invalid ingredients tokens: {len(invalid_ingredients)}")


Invalid directions tokens: 0
Invalid ingredients tokens: 0


In [ ]:
# Check that all values are lists
non_list_directions = df[~df['directions_tokens'].apply(lambda x: isinstance(x, list))]
non_list_ingredients = df[~df['ingredients_tokens'].apply(lambda x: isinstance(x, list))]

print(f"Non-list directions tokens: {len(non_list_directions)}")
print(f"Non-list ingredients tokens: {len(non_list_ingredients)}")


Non-list directions tokens: 0
Non-list ingredients tokens: 0


In [ ]:
# Check for non-integer values in the token lists
invalid_directions_values = df[df['directions_tokens'].apply(lambda x: any(not isinstance(i, int) for i in x))]
invalid_ingredients_values = df[df['ingredients_tokens'].apply(lambda x: any(not isinstance(i, int) for i in x))]

print(f"Invalid directions token values: {len(invalid_directions_values)}")
print(f"Invalid ingredients token values: {len(invalid_ingredients_values)}")


Invalid directions token values: 0
Invalid ingredients token values: 0


In [ ]:
# Check data types
print(df['directions_tokens'].apply(type).value_counts())
print(df['ingredients_tokens'].apply(type).value_counts())


directions_tokens
<class 'list'>    5000
Name: count, dtype: int64
ingredients_tokens
<class 'list'>    5000
Name: count, dtype: int64


In [ ]:
# Inspect the first few tokenized entries to see the output
sample_directions = df['directions'].head().apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
sample_ingredients = df['ingredients'].head().apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

print(f"Sample directions tokens: {sample_directions}")
print(f"Sample ingredients tokens: {sample_ingredients}")


Sample directions tokens: 0    [259, 257, 4334, 362, 36008, 10746, 6839, 5022...
1    [5372, 442, 3949, 12023, 319, 4220, 286, 16871...
2    [259, 257, 3105, 37171, 12082, 477, 9391, 3002...
3    [2127, 346, 290, 1915, 505, 9015, 1234, 13197,...
4    [24011, 500, 717, 1440, 9391, 290, 1803, 287, ...
Name: directions, dtype: object
Sample ingredients tokens: 0    [16, 269, 14245, 11856, 7586, 7543, 352, 14, 1...
1    [16, 1402, 17379, 442, 3949, 12023, 2005, 510,...
2    [17, 1467, 15649, 279, 10025, 12912, 11676, 35...
3    [16, 1588, 2187, 9015, 362, 838, 352, 14, 17, ...
4    [16, 269, 26636, 9215, 513, 14, 19, 269, 7933,...
Name: ingredients, dtype: object


In [ ]:
import torch

# Convert padded token lists to tensors
directions_tensor = torch.tensor(df['directions_tokens'].tolist())
ingredients_tensor = torch.tensor(df['ingredients_tokens'].tolist())

# Check the shape of the tensors
print(directions_tensor.shape)
print(ingredients_tensor.shape)


torch.Size([5000, 512])
torch.Size([5000, 512])


#Trainning Phase

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Combine into a dataset (X, Y)
dataset = TensorDataset(directions_tensor, ingredients_tensor)

# Create DataLoader for batching
batch_size = 8  # You can adjust this based on your system's capacity
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
